In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient
import time

# 환경변수 로드
load_dotenv()

# ==================== 글로벌 언어 설정 ====================
# 여기서 타겟 언어를 변경하세요!
SUPPORTED_LANGUAGES = {
    "한국어": "ko",
    "영어": "en", 
    "중국어(간체)": "zh-Hans",
    "일본어": "ja",
    "독일어": "de",
    "프랑스어": "fr",
    "스페인어": "es",
    "이탈리아어": "it",
    "러시아어": "ru",
    "포르투갈어": "pt",
    "태국어": "th",
    "베트남어": "vi",
    "인도네시아어": "id",
    "말레이어": "ms",
    "힌디어": "hi"
}

# 기본 타겟 언어 설정
DEFAULT_TARGET_LANGUAGE = "한국어"
TARGET_LANGUAGE_CODE = SUPPORTED_LANGUAGES[DEFAULT_TARGET_LANGUAGE]

print(f"🌍 현재 번역 타겟 언어: {DEFAULT_TARGET_LANGUAGE} ({TARGET_LANGUAGE_CODE})")
print(f"📝 지원 언어: {', '.join(SUPPORTED_LANGUAGES.keys())}")

# Azure 설정
vision_endpoint = os.getenv("AZURE_VISION_ENDPOINT")
vision_key = os.getenv("AZURE_VISION_KEY")
translator_key = os.getenv("TRANSLATOR_API_KEY")
translator_region = os.getenv("TRANSLATOR_REGION") 
translator_endpoint = os.getenv("TRANSLATOR_ENDPOINT")

# ==================== OCR + 번역 + 오버레이 클래스 ====================
class VisualTranslator:
    def __init__(self, target_language_code=TARGET_LANGUAGE_CODE):
        # Computer Vision 클라이언트
        self.vision_client = ImageAnalysisClient(
            endpoint=vision_endpoint,
            credential=AzureKeyCredential(vision_key)
        )
        
        # Translator 클라이언트
        self.translator = TextTranslationClient(
            credential=AzureKeyCredential(translator_key),
            endpoint=translator_endpoint,
            region=translator_region
        )
        
        self.target_language = target_language_code
        print(f"✅ Visual Translator 초기화 완료 (목표 언어: {target_language_code})")
    
    def change_target_language(self, language_name):
        """타겟 언어 변경"""
        if language_name in SUPPORTED_LANGUAGES:
            self.target_language = SUPPORTED_LANGUAGES[language_name]
            print(f"🌍 타겟 언어 변경: {language_name} ({self.target_language})")
            return True
        else:
            print(f"❌ 지원하지 않는 언어: {language_name}")
            print(f"지원 언어: {', '.join(SUPPORTED_LANGUAGES.keys())}")
            return False
    
    def extract_text_with_coordinates(self, image_path):
        """OCR로 텍스트와 좌표 추출"""
        try:
            with open(image_path, "rb") as image_data:
                # OCR 수행 (Read API 사용)
                result = self.vision_client.analyze(
                    image_data=image_data.read(),
                    visual_features=[VisualFeatures.READ]
                )
            
            text_regions = []
            if result.read:
                for block in result.read.blocks:
                    for line in block.lines:
                        # 텍스트와 바운딩 박스 추출
                        text = line.text
                        bbox = line.bounding_polygon
                        
                        # 바운딩 박스를 (x, y, width, height) 형태로 변환
                        x_coords = [point.x for point in bbox]
                        y_coords = [point.y for point in bbox]
                        
                        x = min(x_coords)
                        y = min(y_coords)
                        width = max(x_coords) - x
                        height = max(y_coords) - y
                        
                        text_regions.append({
                            'text': text,
                            'bbox': (int(x), int(y), int(width), int(height)),
                            'polygon': [(int(p.x), int(p.y)) for p in bbox]
                        })
            
            print(f"📝 추출된 텍스트 영역: {len(text_regions)}개")
            return text_regions
            
        except Exception as e:
            print(f"❌ OCR 오류: {e}")
            return []
    
    def translate_text(self, text):
        """텍스트 번역"""
        try:
            response = self.translator.translate(
                body=[{"text": text}],
                to_language=[self.target_language]
            )
            
            if response and len(response) > 0:
                translated_text = response['translations']['text']
                detected_language = response.get('detectedLanguage', {}).get('language', 'unknown')
                return translated_text, detected_language
            
            return None, None
            
        except Exception as e:
            print(f"❌ 번역 오류: {e}")
            return None, None
    
    def create_overlay_image(self, original_image_path, text_regions):
        """번역된 텍스트로 오버레이 이미지 생성"""
        # 원본 이미지 로드
        image = cv2.imread(original_image_path)
        overlay = image.copy()
        
        for region in text_regions:
            original_text = region['text']
            bbox = region['bbox']
            
            # 번역
            translated_text, detected_lang = self.translate_text(original_text)
            
            if translated_text:
                print(f"🔄 '{original_text}' → '{translated_text}' ({detected_lang})")
                
                # 원본 텍스트 영역을 반투명 사각형으로 덮기
                x, y, w, h = bbox
                cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 0, 0), -1)
                
                # 번역된 텍스트 추가
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.8
                font_color = (255, 255, 255)
                thickness = 2
                
                # 텍스트 크기 계산
                (text_width, text_height), _ = cv2.getTextSize(translated_text, font, font_scale, thickness)
                
                # 텍스트가 박스에 맞도록 스케일 조정
                if text_width > w:
                    font_scale = font_scale * (w / text_width) * 0.9
                
                # 텍스트 위치 (중앙 정렬)
                text_x = x + (w - text_width) // 2
                text_y = y + (h + text_height) // 2
                
                cv2.putText(overlay, translated_text, (text_x, text_y), 
                           font, font_scale, font_color, thickness)
        
        # 원본과 오버레이 블렌딩
        alpha = 0.7
        result = cv2.addWeighted(image, 1 - alpha, overlay, alpha, 0)
        
        return result
    
    def process_image(self, image_path, output_path=None):
        """전체 이미지 처리 파이프라인"""
        print(f"🖼️ 이미지 처리 시작: {image_path}")
        
        # 1. OCR로 텍스트 추출
        text_regions = self.extract_text_with_coordinates(image_path)
        
        if not text_regions:
            print("❌ 추출된 텍스트가 없습니다.")
            return None
        
        # 2. 번역 및 오버레이 생성
        result_image = self.create_overlay_image(image_path, text_regions)
        
        # 3. 결과 저장/표시
        if output_path:
            cv2.imwrite(output_path, result_image)
            print(f"💾 결과 저장: {output_path}")
        
        return result_image
    
    def capture_and_process_camera(self):
        """카메라에서 스냅샷 촬영 후 처리 (모바일 앱 방식)"""
        print("📷 카메라 스냅샷 + 번역 (SPACE: 촬영, ESC: 종료)")
        
        cap = cv2.VideoCapture(0)
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # 화면에 안내 텍스트 표시
                cv2.putText(frame, "SPACE: Capture & Translate", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, f"Target: {DEFAULT_TARGET_LANGUAGE}", (10, 60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                cv2.imshow('Visual Translator - Camera', frame)
                
                key = cv2.waitKey(1) & 0xFF
                
                # SPACE 키: 스냅샷 촬영 및 처리
                if key == ord(' '):
                    print("📸 스냅샷 촬영 중...")
                    
                    # 스냅샷 저장
                    snapshot_path = f"snapshot_{int(time.time())}.jpg"
                    cv2.imwrite(snapshot_path, frame)
                    
                    # OCR + 번역 처리
                    processed_frame = self.process_image(snapshot_path)
                    
                    if processed_frame is not None:
                        # 결과 표시
                        cv2.imshow('Translated Result', processed_frame)
                        print("번역 결과를 확인하세요. 아무 키나 누르면 닫힙니다...")
                        cv2.waitKey(0)
                        cv2.destroyWindow('Translated Result')
                    
                    # 임시 파일 삭제
                    if os.path.exists(snapshot_path):
                        os.remove(snapshot_path)
                
                # ESC 키: 종료
                elif key == 27:
                    break
                    
        except KeyboardInterrupt:
            print("\n👋 사용자가 중단했습니다.")
        finally:
            cap.release()
            cv2.destroyAllWindows()

# ==================== 언어 선택 메뉴 ====================
def select_target_language():
    """타겟 언어 선택 메뉴"""
    # ✅ global 선언을 함수 맨 처음에 해야 함
    global DEFAULT_TARGET_LANGUAGE, TARGET_LANGUAGE_CODE
    
    print("\n🌍 번역 타겟 언어 선택:")
    print("=" * 40)
    
    languages = list(SUPPORTED_LANGUAGES.keys())
    for i, lang in enumerate(languages, 1):
        current = " ← 현재" if lang == DEFAULT_TARGET_LANGUAGE else ""
        print(f"{i:2}. {lang}{current}")
    
    try:
        choice = int(input(f"\n언어 선택 (1-{len(languages)}) > ").strip())
        if 1 <= choice <= len(languages):
            selected_language = languages[choice-1]
            DEFAULT_TARGET_LANGUAGE = selected_language
            TARGET_LANGUAGE_CODE = SUPPORTED_LANGUAGES[selected_language]
            print(f"✅ 타겟 언어 변경: {selected_language}")
            return True
        else:
            print("❌ 잘못된 번호입니다.")
            return False
    except ValueError:
        print("❌ 숫자를 입력해주세요.")
        return False

# ==================== 메인 실행 ====================
def main():
    print("🎯 Visual Translator - OCR 번역 시스템")
    print("=" * 50)
    print("1. 타겟 언어 선택")
    print("2. 이미지 파일 번역")
    print("3. 카메라 스냅샷 번역 (모바일 앱 방식)")
    print("0. 종료")
    print("=" * 50)
    
    translator = None
    
    while True:
        choice = input("\n선택하세요 > ").strip()
        
        if choice == "1":
            # 타겟 언어 선택
            if select_target_language():
                # 번역기 재초기화 (언어 변경 적용)
                translator = VisualTranslator(TARGET_LANGUAGE_CODE)
        
        elif choice == "2":
            # 이미지 파일 번역
            if translator is None:
                translator = VisualTranslator(TARGET_LANGUAGE_CODE)
            
            image_path = input("이미지 파일 경로 입력 > ").strip()
            if os.path.exists(image_path):
                output_path = f"translated_{os.path.basename(image_path)}"
                result = translator.process_image(image_path, output_path)
                
                if result is not None:
                    print("✅ 번역 완료!")
                    cv2.imshow('Translated Image', result)
                    print("아무 키나 누르면 닫힙니다...")
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
            else:
                print("❌ 파일이 존재하지 않습니다.")
        
        elif choice == "3":
            # 카메라 스냅샷 번역
            if translator is None:
                translator = VisualTranslator(TARGET_LANGUAGE_CODE)
            
            translator.capture_and_process_camera()
        
        elif choice == "0":
            print("👋 프로그램을 종료합니다.")
            break
        
        else:
            print("❌ 잘못된 선택입니다.")

if __name__ == "__main__":
    main()
